<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания 


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

[ваш текст]

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) исользуйтие в проекте коллекции, делегаты, события.


<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [ ]:
using System;
using System.Collections.Generic;

public interface IBankOperations
{
    void Deposit(decimal amount);
    void Withdraw(decimal amount);
    string GetInfo();
}

// Делегат возвращает строку, чтобы соответствовать методу GetInfo
public delegate string DisplayDelegate();

public class BankAccount : IBankOperations
{
    public string AccountNumber { get; set; }
    public decimal Balance { get; protected set; }
    public string AccountType { get; protected set; }

    // Событие изменения баланса
    public event Action<decimal> BalanceChanged;

    public BankAccount(string accountNumber, decimal initialBalance, string accountType)
    {
        AccountNumber = accountNumber;
        Balance = initialBalance;
        AccountType = accountType;
    }

    public virtual void Deposit(decimal amount)
    {
        if (amount <= 0)
            throw new ArgumentException("Сумма пополнения должна быть больше 0.");
        Balance += amount;
        BalanceChanged?.Invoke(Balance);
    }

    public virtual void Withdraw(decimal amount)
    {
        if (amount > Balance)
            throw new InvalidOperationException("Недостаточно средств.");
        Balance -= amount;
        BalanceChanged?.Invoke(Balance);
    }

    public virtual string GetInfo()
    {
        return $"Номер счета: {AccountNumber}, Баланс: {Balance:C}, Тип счета: {AccountType}";
    }
}

public class SavingsAccount : BankAccount
{
    public decimal InterestRate { get; set; }

    public SavingsAccount(string accountNumber, decimal initialBalance, string accountType, decimal interestRate)
        : base(accountNumber, initialBalance, accountType)
    {
        InterestRate = interestRate;
    }

    public override void Deposit(decimal amount)
    {
        base.Deposit(amount);
        Balance += Balance * (InterestRate / 100); // Начисление процентов
    }

    public override string GetInfo()
    {
        return base.GetInfo() + $", Процентная ставка: {InterestRate}%";
    }
}

public class SecureDepositAccount : SavingsAccount, IBankOperations
{
    public bool IsAccountBlocked { get; private set; }

    public SecureDepositAccount(string accountNumber, decimal initialBalance, string accountType, decimal interestRate)
        : base(accountNumber, initialBalance, accountType, interestRate)
    {
        IsAccountBlocked = false;
    }

    void IBankOperations.Deposit(decimal amount)
    {
        if (IsAccountBlocked)
        {
            throw new InvalidOperationException("Счет заблокирован. Внесение средств невозможно.");
        }
        base.Deposit(amount);
    }

    void IBankOperations.Withdraw(decimal amount)
    {
        if (IsAccountBlocked)
        {
            throw new InvalidOperationException("Счет заблокирован. Снятие средств невозможно.");
        }
        base.Withdraw(amount);
    }

    string IBankOperations.GetInfo()
    {
        string blockedStatus = IsAccountBlocked ? "Заблокирован" : "Активен";
        return base.GetInfo() + $", Статус счета: {blockedStatus}";
    }

    public void BlockAccount()
    {
        IsAccountBlocked = true;
        Console.WriteLine($"Счет {AccountNumber} заблокирован.");
    }
}

public class AccountManager
{
    private readonly IBankOperations _bankOperations;

    public AccountManager(IBankOperations bankOperations)
    {
        _bankOperations = bankOperations;
    }

    public void MakeDeposit(decimal amount)
    {
        _bankOperations.Deposit(amount);
        Console.WriteLine($"Пополнение в размере {amount:C} успешно выполнено.");
    }

    public void MakeWithdrawal(decimal amount)
    {
        _bankOperations.Withdraw(amount);
        Console.WriteLine($"Снятие в размере {amount:C} успешно выполнено.");
    }

    public void PrintAccountInfo()
    {
        Console.WriteLine(_bankOperations.GetInfo());
    }
}


        // Создаем счета
        var secureAccount = new SecureDepositAccount("98765", 2000, "Депозит", 2.5m);
        var savingsAccount = new SavingsAccount("12345", 1500, "Сберегательный", 3.0m);

        // Подписываемся на события изменения баланса
        secureAccount.BalanceChanged += newBalance => Console.WriteLine($"SecureAccount новый баланс: {newBalance:C}");
        savingsAccount.BalanceChanged += newBalance => Console.WriteLine($"SavingsAccount новый баланс: {newBalance:C}");

        // Используем делегат для отображения информации
        DisplayDelegate displaySecure = secureAccount.GetInfo;
        DisplayDelegate displaySavings = savingsAccount.GetInfo;

        Console.WriteLine("Информация о счетах:");
        Console.WriteLine(displaySecure.Invoke()); // Вызов делегата
        Console.WriteLine(displaySavings.Invoke());

        var manager = new AccountManager(savingsAccount);
        manager.MakeDeposit(500);
        manager.MakeWithdrawal(200);
        manager.PrintAccountInfo();

        // Лямбда-выражения для фильтрации
        List<BankAccount> accounts = new List<BankAccount> { secureAccount, savingsAccount };
        var highBalanceAccounts = accounts.FindAll(acc => acc.Balance > 1000);

        Console.WriteLine("\nСчета с балансом более 1000:");
        foreach (var acc in highBalanceAccounts)
        {
            Console.WriteLine(acc.GetInfo());
        }

Информация о счетах:
Номер счета: 98765, Баланс: ¤2,000.00, Тип счета: Депозит, Процентная ставка: 2.5%
Номер счета: 12345, Баланс: ¤1,500.00, Тип счета: Сберегательный, Процентная ставка: 3.0%
SavingsAccount новый баланс: ¤2,000.00
Пополнение в размере ¤500.00 успешно выполнено.
SavingsAccount новый баланс: ¤1,860.00
Снятие в размере ¤200.00 успешно выполнено.
Номер счета: 12345, Баланс: ¤1,860.00, Тип счета: Сберегательный, Процентная ставка: 3.0%

Счета с балансом более 1000:
Номер счета: 98765, Баланс: ¤2,000.00, Тип счета: Депозит, Процентная ставка: 2.5%
Номер счета: 12345, Баланс: ¤1,860.00, Тип счета: Сберегательный, Процентная ставка: 3.0%


In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;

public enum OrderStatus
{
    Created,
    InProgress,
    Completed
}

public class Customer
{
    public string FirstName { get; set; }
    public string LastName { get; set; }
    public string PhoneNumber { get; set; }

    public Customer(string firstName, string lastName, string phoneNumber)
    {
        FirstName = firstName;
        LastName = lastName;
        PhoneNumber = phoneNumber;
    }
}

public class Employee
{
    public string FirstName { get; set; }
    public string LastName { get; set; }
    public string Position { get; set; }
    public int OrdersProcessed { get; private set; }

    public Employee(string firstName, string lastName, string position)
    {
        FirstName = firstName;
        LastName = lastName;
        Position = position;
        OrdersProcessed = 0;
    }

    public void ProcessOrder()
    {
        OrdersProcessed++;
    }
}

public class Order
{
    public Guid OrderId { get; }
    public string Description { get; set; }
    public Employee AssignedEmployee { get; set; }
    public Customer Customer { get; set; }
    public OrderStatus Status { get; private set; }
    public DateTime CreatedAt { get; }

    public Order(string description, Customer customer)
    {
        OrderId = Guid.NewGuid();
        Description = description;
        Customer = customer;
        Status = OrderStatus.Created;
        CreatedAt = DateTime.Now;
    }

    public event Action<Order> StatusChanged;

    public void UpdateStatus(OrderStatus newStatus)
    {
        Status = newStatus;
        StatusChanged?.Invoke(this);
    }
}

public class Company
{
    public string Name { get; set; }
    public List<Employee> Employees { get; } = new List<Employee>();
    public List<Order> Orders { get; } = new List<Order>();

    public Company(string name)
    {
        Name = name;
    }

    public void AddEmployee(Employee employee)
    {
        Employees.Add(employee);
    }

    public void AddOrder(Order order)
    {
        Orders.Add(order);
    }

    public void AssignEmployeeToOrder(Order order, Employee employee)
    {
        order.AssignedEmployee = employee;
    }

    public List<Order> GetOrdersByStatus(OrderStatus status)
    {
        return Orders.Where(o => o.Status == status).ToList();
    }

    public List<Order> GetOrdersByCustomer(Customer customer)
    {
        return Orders.Where(o => o.Customer == customer).ToList();
    }

    public void GenerateEmployeeReport()
    {
        Console.WriteLine("Employee Report:");
        foreach (var employee in Employees)
        {
            Console.WriteLine($"- {employee.FirstName} {employee.LastName}: {employee.OrdersProcessed} заказ обработан");
        }
    }
}

// Пример использования
var company = new Company("Технологические Решения");

var emp1 = new Employee("Илья", "Кайдалов", "Глав. Менеджер");
var emp2 = new Employee("Эллейн", "Буртовая", "ООП разработчик");
company.AddEmployee(emp1);
company.AddEmployee(emp2);

var customer1 = new Customer("Кирилл", "Андреев", "783-4566-7890");
var customer2 = new Customer("Боб", "Кидалов", "478-345-4111");

var order1 = new Order("Разработка виртуальных окружений", customer1);
var order2 = new Order("Исправление ошибок в разработанном коде", customer2);

order1.StatusChanged += order =>
{
    Console.WriteLine($"Заказ {order.OrderId} изменил статус на {order.Status}");
};

order2.StatusChanged += order =>
{
    Console.WriteLine($"Заказ {order.OrderId} изменил статус на {order.Status}");
};

company.AddOrder(order1);
company.AddOrder(order2);

company.AssignEmployeeToOrder(order1, emp1);
company.AssignEmployeeToOrder(order2, emp2);

order1.UpdateStatus(OrderStatus.InProgress);
emp1.ProcessOrder();
order2.UpdateStatus(OrderStatus.Completed);
emp2.ProcessOrder();
company.GenerateEmployeeReport();

Console.WriteLine("\nЗаказы в работе:");
foreach (var order in company.GetOrdersByStatus(OrderStatus.InProgress))
{
    Console.WriteLine($"- Заказ {order.OrderId}: {order.Description}");
}

Заказ 63f9e0cd-7df0-4236-8148-9c34ac89fe5d изменил статус на InProgress
Заказ 469a9105-37ea-40b7-970a-551b018a95d5 изменил статус на Completed
Employee Report:
- Илья Кайдалов: 1 заказ обработан
- Эллейн Буртовая: 1 заказ обработан

Заказы в работе:
- Заказ 63f9e0cd-7df0-4236-8148-9c34ac89fe5d: Разработка виртуальных окружений
